In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from TextureClassifier import TextureClassifier
import os
import logging
import numpy as np
import socket
hostname = socket.gethostname()

import sys
import mxnet as mx

In [3]:
logging.basicConfig(level=logging.DEBUG)

# stack = 'MD589'
# sec = 165

stack = 'MD594'
sec = 289

if hostname.startswith('gcn'):
    data_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/%s_patches/%04d_reduced'%(stack, sec)
    model_dir = '/oasis/projects/nsf/csd395/yuncong/jiaxu_repo/model_publish'    
elif hostname == 'yuncong-MacbookPro':
#     data_dir = '/home/yuncong/jiaxu_sample_patches'
    data_dir = '/home/yuncong/CSHL_data_patches/%s_patches/%04d_reduced'%(stack, sec)
    model_dir = '/home/yuncong/jiaxu_repo/model_publish' 
    
# p_labels = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

prefix = 'inception-stage1'
n_iter = 6

In [15]:
img_filenames = sorted(os.listdir(data_dir))
with open('/home/yuncong/CSHL_data_patches/%s_%04d_test.lst'%(stack, sec), 'w') as lst_file:
    for index, img_filename in enumerate(img_filenames[:128]):
        lst_file.write('%s\t0\t%s\n'%(index, img_filename))

In [16]:
os.system(os.environ['MXNET_DIR']+'/bin/im2rec /home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.lst \
/home/yuncong/CSHL_data_patches/%(stack)s_patches/%(sec)04d_reduced/ \
/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.rec' % \
          {'stack': stack,
           'sec': sec})
# on mbp, scanning 1000 images take 1.8s
# on gordon, scanning 16 image, 0.06s

0

In [19]:
# if the dataset has more than one batch's amount of data, then you can see effect of multithreading enabled by openblas

import time
t = time.time()

batch_size = 16

if hostname.startswith('gcn'):
#     init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=[mx.cpu(i) for i in range(16)])
    init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.cpu())
    # [mx.cpu(i) for i in range(16)] vs. mx.cpu() makes no difference
else:
    init_model = mx.model.FeedForward.load(os.path.join(model_dir, prefix), n_iter, ctx=mx.gpu())

print time.time() - t

t = time.time()

test_iter = mx.io.ImageRecordIter(
    path_imgrec='/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test.rec' % {'stack': stack, 'sec':sec},
    batch_size=batch_size,
    data_shape=(3, 224, 224),
    mean_img=os.path.join(model_dir, 'mean_224.nd'),
)

print time.time() - t

t = time.time()

probs = init_model.predict(test_iter)

print time.time() - t

0.0438940525055
0.137769937515
63.7382729053


In [ ]:
# on Gordon, predicting 1 patch takes .5 s; OPENBLAS_NUM_THREADS=16 but CPUs are not fully utilized
# on mbp, predicting 16 patches takes 1 s

In [9]:
# prediction_map = zip(np.argmax(probs, axis=1), map(lambda x: int(x[:-4]), img_filenames))
np.save('/home/yuncong/CSHL_data_patches/%(stack)s_%(sec)04d_test_prediction_map.npy'% {'stack': stack, 'sec':sec}, 
        probs)